In [73]:
from dotenv import load_dotenv
import os

In [74]:
# Load variables from .env into environment
load_dotenv()

# Access them with os.environ
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")
debug_mode = os.getenv("DEBUG") == "True"

paper = True
data_api_url = None

Alpaca API lets you trade stocks programmatically. Below is a simple example of how to use the Alpaca API with Python to place a buy order for a stock.


In [75]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

pd.set_option("display.max_rows", 500)

In [76]:
stock_historical_data_client = StockHistoricalDataClient(
    api_key, secret_key, url_override=data_api_url
)

symbol = "GOOGL"

# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/Chicago"))
req = StockBarsRequest(
    symbol_or_symbols=[symbol],
    timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),  # specify timeframe
    start=now
    - timedelta(
        weeks=52 * 4
    ),  # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    # limit=1000,  # specify limit
)

df_bars = stock_historical_data_client.get_stock_bars(req).df

# Calculate 5-period and 20-period Simple Moving Average (SMA)
df_bars["SMA_5"] = df_bars["close"].rolling(window=5).mean()
df_bars["SMA_20"] = df_bars["close"].rolling(window=20).mean()

# Exclude days where SMA values are NaN
df_bars = df_bars.dropna(subset=["SMA_5", "SMA_20"])

# Convert timestamp to date in New York timezone and set as new index
df_bars = df_bars.sort_index(level=["symbol", "timestamp"])


# timestamps_utc = df_bars.index.get_level_values("timestamp")
# dates_ny = timestamps_utc.tz_convert("America/New_York").date
# symbols = df_bars.index.get_level_values("symbol")
# df_bars.index = pd.MultiIndex.from_arrays([symbols, dates_ny], names=["symbol", "date"])

print(f"Number of rows after dropping NaNs: {len(df_bars)}")
display(df_bars.tail(3))

Number of rows after dropping NaNs: 981


open     high     low   close      volume  \
symbol timestamp                                                                
GOOGL  2025-09-10 04:00:00+00:00  238.90  241.660  237.85  239.17  35141074.0   
       2025-09-11 04:00:00+00:00  239.88  242.250  236.25  240.37  30593309.0   
       2025-09-12 04:00:00+00:00  240.37  242.075  238.00  240.80  26771610.0   

                                  trade_count        vwap    SMA_5    SMA_20  
symbol timestamp                                                              
GOOGL  2025-09-10 04:00:00+00:00     514894.0  239.713609  236.028  214.4420  
       2025-09-11 04:00:00+00:00     457661.0  239.741396  237.642  216.3625  
       2025-09-12 04:00:00+00:00     431474.0  240.395020  238.802  218.2555

## Check Conditions for Trading Signals


### Closing > Opening


In [77]:
df_bars["is_bullish"] = df_bars["close"] > df_bars["open"]
df_bars["is_bullish"].head(3)

symbol  timestamp                
GOOGL   2021-10-15 04:00:00+00:00    False
        2021-10-18 04:00:00+00:00     True
        2021-10-19 04:00:00+00:00    False
Name: is_bullish, dtype: bool

In [78]:
def check_initial_buy_conditions(df_window, current_row):
    """
    Checks conditions for opening a new position.
    - Candlestick is bullish.
    - Closing price is the lowest in the last 30 bullish candlesticks.
    """
    if not current_row["is_bullish"]:
        return False

    bullish_window = df_window[df_window["is_bullish"]]
    if bullish_window.empty:
        return True  # If no other bullish candles, this is the lowest.

    return current_row["close"] <= bullish_window["close"].min()


def check_add_to_position_conditions(current_row, group):
    """
    Checks conditions for adding to an existing position.
    - Candlestick is bullish.
    - Closing price is at least 1% lower than the group's average purchase price.
    """
    if not current_row["is_bullish"]:
        return False

    return current_row["close"] <= group["average_price"] * 0.99


def check_sell_conditions(current_row, group):
    """
    Checks conditions for selling all shares in a group.
    - Closing price is at least 3% higher than the group's average purchase price.
    """
    return current_row["close"] >= group["average_price"] * 1.03

In [ ]:
# --- Simulation ---
active_groups = []
next_group_id = 1
reusable_group_ids = []  # Keep track of sold group IDs
trades = []
cash = 100000  # Starting cash

for i in range(29, len(df_bars)):
    df_window = df_bars.iloc[i - 29 : i]
    current_row = df_bars.iloc[i]
    date = current_row.name[1]

    # --- Process existing groups (check for sell or add) ---
    groups_to_remove = []
    for group in active_groups:
        # Check for sell condition
        if check_sell_conditions(current_row, group):
            cash += group["shares"] * current_row["close"]
            trades.append(
                {
                    "date": date,
                    "type": "sell",
                    "price": current_row["close"],
                    "shares": group["shares"],
                    "cash": cash,
                    "group_id": group["id"],
                }
            )
            groups_to_remove.append(group)
            reusable_group_ids.append(group["id"])  # Add group ID for reuse
        # Check for add condition (if group is not full)
        elif group["shares"] < 3 and check_add_to_position_conditions(
            current_row, group
        ):
            cash -= current_row["close"]
            new_total_cost = (
                group["average_price"] * group["shares"] + current_row["close"]
            )
            group["shares"] += 1
            group["average_price"] = new_total_cost / group["shares"]
            trades.append(
                {
                    "date": date,
                    "type": "add",
                    "price": current_row["close"],
                    "shares": 1,
                    "cash": cash,
                    "group_id": group["id"],
                }
            )
            # Stop adding after one group is added to for the day
            break

    # Remove sold groups and sort reusable IDs
    if groups_to_remove:
        active_groups = [g for g in active_groups if g not in groups_to_remove]
        reusable_group_ids.sort()

    # --- Check for initial buy condition to open a new group ---
    all_groups_full = all(g["shares"] >= 3 for g in active_groups)
    if not active_groups or all_groups_full:
        if check_initial_buy_conditions(df_window, current_row):
            cash -= current_row["close"]

            # Determine the group ID to use
            if reusable_group_ids:
                group_id_to_use = reusable_group_ids.pop(0)
            else:
                group_id_to_use = next_group_id
                next_group_id += 1

            new_group = {
                "id": group_id_to_use,
                "shares": 1,
                "average_price": current_row["close"],
            }
            active_groups.append(new_group)
            trades.append(
                {
                    "date": date,
                    "type": "buy",
                    "price": current_row["close"],
                    "shares": 1,
                    "cash": cash,
                    "group_id": group_id_to_use,
                }
            )


df_trades = pd.DataFrame(trades)
if not df_trades.empty:
    df_trades = df_trades.set_index("date")

print("--- Simulation Results ---")
print(f"Final cash: ${cash:,.2f}")
if active_groups:
    print(
        f"Holding {sum(g['shares'] for g in active_groups)} shares across {len(active_groups)} groups."
    )
    for group in active_groups:
        print(
            f"  - Group {group['id']}: {group['shares']} shares at avg price ${group['average_price']:,.2f}"
        )

print("\n--- Trades ---")
display(df_trades)

--- Simulation Results ---
Final cash: $94,607.15
Holding 3 shares across 1 groups.
  - Group 4: 3 shares at avg price $2,387.86

--- Trades ---


,type,price,shares,cash,group_id
date,,,,,
2021-12-20 05:00:00+00:00,buy,2832.14,1,97167.86,1
2021-12-22 05:00:00+00:00,sell,2928.30,1,100096.16,1
2022-01-06 05:00:00+00:00,buy,2754.95,1,97341.21,2
2022-01-24 05:00:00+00:00,add,2616.08,1,94725.13,2
2022-02-02 05:00:00+00:00,sell,2960.00,2,100645.13,2
2022-03-08 05:00:00+00:00,buy,2542.09,1,98103.04,3
2022-03-09 05:00:00+00:00,sell,2668.40,1,100771.44,3
2022-04-25 04:00:00+00:00,buy,2461.48,1,98309.96,4
2022-04-28 04:00:00+00:00,add,2370.45,1,95939.51,4
